In [1]:
import numpy          as np
from scipy.spatial    import ConvexHull
from shapely          import geometry
from geopandas        import GeoDataFrame
from gasp3.dt.fm      import tbl_to_obj
from gasp3.gt.mng.prj import project
from gasp3.dt.to      import obj_to_tbl
from gasp3.dt.to.geom import pnt_dfwxy_to_geodf
from gasp3.dt.to.shp  import df_to_shp

In [2]:
contributions = r'D:\indev\fireloc\tst_places.shp'

real_fire = r'D:\indev\fireloc\tst_fire.shp'

result = r'D:\indev\fireloc\tst_real_compass.xlsx'

declinacao = -(7 + (36/60.0))

col_direction = 'REAL_DIREC'

pnt_result  = r'D:\indev\fireloc\fire_compass.shp'
poly_result = None

In [3]:
pnts = tbl_to_obj(contributions)

pnts = pnts[[col_direction, 'geometry']]

pnts = project(pnts, None, 20790, inEPSG=3763, gisApi='pandas')

pnts['x_coord'] = pnts.geometry.x
pnts['y_coord'] = pnts.geometry.y
pnts['relatio'] = 1
pnts['fid']     = pnts.index

pnts = pnts.merge(pnts, how='outer', left_on='relatio', right_on='relatio')
pnts = pnts[pnts.fid_x < pnts.fid_y]

pnts.drop(['geometry_x', 'geometry_y', 'relatio'], axis=1, inplace=True)

pnts.rename(columns={
    col_direction + '_x' : 'direcao_a_q', col_direction + '_y' : 'direcao_b_q',
    'x_coord_x' : 'a_x', 'y_coord_x' : 'a_y', 'x_coord_y' : 'b_x', 'y_coord_y' : 'b_y'
}, inplace=True)

pnts['direcao_a_q'] = pnts['direcao_a_q'] + declinacao
pnts['direcao_b_q'] = pnts['direcao_b_q'] + declinacao

In [4]:
pnts.head(100)

,direcao_a_q,a_x,a_y,fid_x,direcao_b_q,b_x,b_y,fid_y
1,262.4,175299.374999,359946.587506,0,242.4,175291.625504,360080.772765,1
2,262.4,175299.374999,359946.587506,0,152.4,174935.191573,360083.350713,2
3,262.4,175299.374999,359946.587506,0,42.4,174929.750502,359906.735751,3
4,262.4,175299.374999,359946.587506,0,252.4,175145.370666,359966.837703,4
5,262.4,175299.374999,359946.587506,0,262.4,175149.235143,359951.157044,5
8,242.4,175291.625504,360080.772765,1,152.4,174935.191573,360083.350713,2
9,242.4,175291.625504,360080.772765,1,42.4,174929.750502,359906.735751,3
10,242.4,175291.625504,360080.772765,1,252.4,175145.370666,359966.837703,4
11,242.4,175291.625504,360080.772765,1,262.4,175149.235143,359951.157044,5
15,152.4,174935.191573,360083.350713,2,42.4,174929.750502,359906.735751,3


In [5]:
def pd_direct_intersection(df, resCol, pntAX, pntAY, pntBX, pntBY, dir_aq, dir_bq):
    resCol_P = resCol + '_P'
    resCol_M = resCol + '_M'
    
    df[resCol_P] = (
        (df[pntBX] - df[pntAX]) - (df[pntBY] - df[pntAY]) * np.tan(
            np.radians(df[dir_bq]))
    ) / (
        np.tan(np.radians(df[dir_aq])) - np.tan(np.radians(df[dir_bq]))
    ) + df[pntAY]
    
    df[resCol_M] = (
        (df[resCol_P] - df[pntAY]) * np.tan(np.radians(df[dir_aq]))
    ) + df[pntAX]
    
    return df

pnts = pd_direct_intersection(
    pnts, 'q', 'a_x', 'a_y', 'b_x', 'b_y',
    'direcao_a_q', 'direcao_b_q'
)

In [6]:
def pd_lateral_intersection(df, resCol, pntAX, pntAY, pntBX, pntBY, dir_aq, dir_bq):
    resCol_P = resCol + '_P'
    resCol_M = resCol + '_M'
    
    df[resCol_M] = (
        (df[pntBY] - df[pntAY]) - (df[pntBX] - df[pntAX]) * (
            np.cos(np.radians(df[dir_bq])) / np.sin(np.radians(df[dir_bq]))
        )
    ) / (
        (
            np.cos(np.radians(df[dir_aq])) / np.sin(np.radians(df[dir_aq]))
        ) - (
            np.cos(np.radians(df[dir_bq])) / np.sin(np.radians(df[dir_bq]))
        )
    ) + df[pntAX]
    
    df[resCol_P] = ((df[resCol_M] - df[pntAX]) * (
        np.cos(np.radians(df[dir_aq])) / np.sin(np.radians(df[dir_aq]))
    )) + df[pntAY]
    
    return df

pnts = pd_lateral_intersection(
    pnts, 'l_q', 'a_x', 'a_y', 'b_x', 'b_y',
    'direcao_a_q', 'direcao_b_q'
)

(15, 12)


In [7]:
pnts['q_P'] = np.where((pnts.q_P == np.inf) | (pnts.q_P == -np.inf), pnts.l_q_P, pnts.q_P)
pnts['q_M'] = np.where((pnts.q_M == np.inf) | (pnts.q_M == -np.inf), pnts.l_q_M, pnts.q_M)
pnts.drop(['l_q_P', 'l_q_M'], axis=1, inplace=True)

In [8]:
print(pnts.shape)
pnts = pnts[~pnts.isin([np.nan, np.inf, -np.inf]).any(1)]
print(pnts.shape)

(15, 10)
(14, 10)


In [9]:
hull = ConvexHull(pnts[['q_M', 'q_P']])

poly = geometry.Polygon([[
    pnts.q_M.iloc[idx], pnts.q_P.iloc[idx]
] for idx in hull.vertices])

convex_df = GeoDataFrame(
    [1], columns=['cat'],
    crs={'init' : 'epsg:' + str(20790)},
    geometry=[poly]
)

In [10]:
convex_df.head(5)

,cat,geometry
0,1,"POLYGON ((174991.9263834366 359974.8270302032,..."


In [11]:
pnts = pnt_dfwxy_to_geodf(pnts, 'q_M', 'q_P', 20790)
pnts = project(pnts, None, 3763, inEPSG=20790, gisApi='pandas')

convex_df = project(convex_df, None, 3763, inEPSG=20790, gisApi='pandas')

df_to_shp(pnts, pnt_result)
if not poly_result:
    import os
    from gasp3.pyt.oss import get_filename
    
    poly_result = os.path.join(os.path.dirname(pnt_result), get_filename(pnt_result) + '_convex.shp')

df_to_shp(convex_df, poly_result)

'D:\\indev\\fireloc\\fire_compass_convex.shp'

In [ ]:
def direct_intersection(pntA, pntB, d_ax, d_bx):
    import math
    
    pnt_a_x, pnt_a_y = pntA
    pnt_b_x, pnt_b_y = pntB
    
    pq_num = (pnt_b_x - pnt_a_x) - (
        pnt_b_y - pnt_a_y) * math.tan(math.radians(d_bx))
    
    pq_den = math.tan(math.radians(d_ax)) - math.tan(math.radians(d_bx))
    
    pq = (pq_num / pq_den) + pnt_a_y
    
    mq = ((pq - pnt_a_y) * math.tan(math.radians(d_ax))) + pnt_a_x
    
    return mq, pq

def lateral_intersection(pntA, pntB, d_ax, d_bx):
    import math
    
    pnt_a_x, pnt_a_y = pntA
    pnt_b_x, pnt_b_y = pntB
    
    mq_num = (pnt_b_y - pnt_a_y) - (pnt_b_x - pnt_a_x) * (
        math.cos(math.radians(d_bx)) / math.sin(math.radians(d_bx)))
    
    mq_den = (
        math.cos(math.radians(d_ax)) / math.sin(math.radians(d_ax))
    ) - (
        math.cos(math.radians(d_bx)) / math.sin(math.radians(d_bx))
    )
    
    mq = (mq_num / mq_den) + pnt_a_x
    
    pq = ((mq-pnt_a_x) * (
        math.cos(math.radians(d_ax)) / math.sin(math.radians(d_ax))
    )) + pnt_a_y
    
    return mq, pq